### Mudule extract data from pdf file

In [10]:
import io
import os
import sys
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
from pdf2image import convert_from_path


In [11]:
print(os.getcwd())
print(sys.argv[0])
print(os.path.dirname(os.path.realpath('__file__')))

/Users/johnlennon/My Drive/Python/Doc_mentor/Doc_mentor
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/ipykernel_launcher.py
/Users/johnlennon/My Drive/Python/Doc_mentor/Doc_mentor


In [12]:
# file directory
work_file_dir = os.path.dirname(os.path.realpath('__file__'))
# WORK_DIR = os.getcwd()

# Check pdf absolute path
PDFs_PATH = os.path.join(work_file_dir, 'data/')
# OUT_PATH = os.path.join(work_file_dir, 'data/202109_bill_rec.jpg')
# Relative path
PDF_FILE = 'data/202109_bill_rec.pdf'
# OUTPUT_FILE = 'data/simple_bill.txt'

In [13]:
# Variant 1
### We will use 'pdfminer' library and https://python-scripts.com/exporting-data-from-pdf

# bills_names = ['Счет', 'Счёт', 'СЧЕТ', 'С Ч Ё Т', 'СЧЁТ', 'END']
# upd_names = ['Фактура', 'END']
# act_names = ['Акт', 'END']
# pack_names = ['Товарная накладная', 'ТОВАРНАЯ НАКЛАДНАЯ', 'END']


# WE_LOOK = upd_names

# def extract_text_by_page(PDF_FILE):
#     with open(PDF_FILE, 'rb') as fh:
#             for page in PDFPage.get_pages(fh, 
#                                   caching=True,
#                                   check_extractable=True):
#             resource_manager = PDFResourceManager()
#             fake_file_handle = io.StringIO()
#             converter = TextConverter(resource_manager, fake_file_handle)
#             page_interpreter = PDFPageInterpreter(resource_manager, converter)
#             page_interpreter.process_page(page)

#             text = fake_file_handle.getvalue()
#             yield text

#             # close open handles
#             converter.close()
#             fake_file_handle.close()

# def extract_text(PDF_FILE):
#     """ check the pdf file for pages with lookup values and return list of pages"""
#     i = 0
#     pages_list = []
#     for page in extract_text_by_page(PDF_FILE):
#         for bill_feature in WE_LOOK:
#             if(page.find(bill_feature) != -1):
#                 pages_list.append(i)
#                 break
#         if (bill_feature == 'END') and (len(page) > 24):
#             print(f'На странице {i} не обнаружен {WE_LOOK[0]}', page, '\n')
        
#         i += 1
#     return pages_list

# def extract_pages_to_jpg(PDF_FILE):
#     """extract jpg files from pdf"""
#     checked_pages = extract_text(PDF_FILE)       
#     pages = convert_from_path(PDF_FILE, 200)
#     i = 0
#     for page in pages:
#         print(i)
#         if i in checked_pages:
#             page.save(f'{PDF_FILE[:-4]}_{i}.jpg', 'JPEG')
#         i += 1
#     print(f'Extracted pages:{checked_pages}')

# extract_pages_to_jpg(PDF_FILE)        

In [14]:
# Variant 2
### We will use 'pdfminer' library and https://python-scripts.com/exporting-data-from-pdf


bills_names = ['Счет', 'Счёт', 'СЧЕТ', 'С Ч Ё Т', 'СЧЁТ', 'END']
upd_names = ['Фактура', 'фактура','END']
act_names = ['Акт', 'END']
pack_names = ['Товарная накладная', 'ТОВАРНАЯ НАКЛАДНАЯ', 'END']
doc_dict = {'pl':pack_names, 'act': act_names, 'upd': upd_names} # 'bill':bills_names
doc_dict_bill = {'pl':pack_names, 'act': act_names, 'bill':bills_names} # 'bill':bills_names
JPG_PPI = 100

WE_LOOK = doc_dict

def extract_text_by_page(PDF_FILE_NAME, path='data/'):
    
    
    for key in doc_dict.keys():
        try:
            os.mkdir(path+key)
        except:
            pass
#             print(f'Directory {key} was made already')
        
    extr_pages_dict = {}
    i = 0
    full_pdf_path = os.path.join(path, PDF_FILE_NAME)
    with open(full_pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
            resource_manager = PDFResourceManager()
            fake_file_handle = io.StringIO()
            converter = TextConverter(resource_manager, fake_file_handle)
            page_interpreter = PDFPageInterpreter(resource_manager, converter)
            page_interpreter.process_page(page)
            text = fake_file_handle.getvalue()
            #lookup matches with features
            notfnd = True
            for key, names in WE_LOOK.items():
                if notfnd:
                    for name in names:
                        if(text.find(name) != -1):
                            extr_pages_dict.update({i:key})
                            notfnd = False
                            break
                else:
                    break
            i += 1
#             yield text

            # close open handles
            converter.close()
            fake_file_handle.close()
#     print(f'Dictionary formed {extr_pages_dict}')
    # extract to jpg
    pages = convert_from_path(full_pdf_path, 100)
    i = 0
    for page in pages:
        if i in extr_pages_dict.keys():
            doc_type = extr_pages_dict[i]
            out_file_name = f'{PDF_FILE_NAME[:-4]}_{doc_type}_{i}.jpg'
            out_file = f'{path}/{doc_type}/{out_file_name}'
            try:
                page.save(out_file, 'JPEG')
            except:
                print(f'file {out_file} not saved')
#             print(f'Page {i} from {PDF_FILE_NAME} was extracted to {out_file_name}')
        i += 1
    return True
  

In [15]:
# Scan dir PDFs_PATH and put file names to list
def pdf_files_list(PDFs_PATH, type='abs'):
    """return list of pdf files absolute or relative"""
    # put all files
    PDF_FILE_LIST = [os.path.join(PDFs_PATH, name) 
                     for name in os.listdir(PDFs_PATH) 
                     if os.path.isfile(os.path.join(PDFs_PATH, name))]
    # select only .pdf files
    PDF_FILE_LIST = [elem for elem in PDF_FILE_LIST if elem[-4:]=='.pdf']
    PDF_FILE_LIST_ABS = [elem.split('/')[-1] for elem in PDF_FILE_LIST]
    if type=='abs':
        return PDF_FILE_LIST_ABS
    else:
        return PDF_FILE_LIST
    

In [17]:
for file in pdf_files_list(PDFs_PATH):
    print(f'File {file} start extract!')
    extract_text_by_page(file, path=PDFs_PATH)
    print(f'File file {file} was extracted!')
print('Finished!')

File 201905_bill_rec.pdf start extract!
File file 201905_bill_rec.pdf was extracted!
Finished!
